In [ ]:
from py2neo import Graph, Node, Relationship
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Connect to graph
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password")) 

In [ ]:
#Retrieve all tweets mentioning candidates
query = """\
MATCH p= (c:Candidate)<-[]-(t:Tweet)
return t.created_at as time, c.screen_name as candidate, t.sentiment as sentiment
"""
s_df=graph.run(query).to_data_frame()
s_df.set_index(pd.to_datetime(df['time']).timestamp())

In [ ]:
#Make sure things look alright
s_df.head()

In [ ]:
#Create a sentiment plot over time for each candidate
for candidate in df.candidate:
    plt.figure()
    sns.distplot(s_df.loc[candidate].query[one hour before debate<df.index<one hour after debate]['sentiment'])
    plt.title(candidate)
    plt.xlim((-1,1))
    tweet_cnt = len(s_df.loc[candidate].query[one hour before debate<df.index<one hour after debate]['sentiment'])
    plt.xlabel(f'Tweet Sentiment for {tweet_cnt} tweets')
    plt.show();

In [ ]:
#Retrieve updates on candidate follower counts over time
retweet_sample_query= """
MATCH (c:Candidate)<-[rt]-[u:User]
RETURN c.name, rt.usrFollowerCount as followers, r.timestamp as time
"""
rt_df=graph.run(retweet_time_query).to_data_frame()

In [ ]:
#Create plot showing follower change for each candidate
for ind in range(len(rt_df)):
    fig,ax = plt.subplots()
    tweet = rt_df.iloc[ind]
    min_t=min(tweet['time'])
    min_f=min(tweet['followers'])
    t = (pd.Series(tweet['time'])-min_t)/60/60
    name = tweet['c.name']
    fol = pd.Series(tweet['followers'])-min_f
    plt.plot(t,fol)
    plt.title(name)
    plt.xlabel('Time since the morning of the debate')
    plt.legend(['Follower Change'])
    plt.show();